In [26]:
#Imports
import sklearn
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

The overall plan is to:
1. Run lookup() - which finds which song data to use based on a trackid json payload
2. Run predict() - Preprocess the selected song and find k nearest neighbors' track ids
3. Run deliver() - Takes k track ids from predict, finds the associated track ids, then sends it off as a json payload.

In [17]:
#Read in DF
songs = pd.read_csv("song_list.csv")
features = ['acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence',
       'popularity']
print(f'This dataframe looks like {songs.shape} and has features of: {features}')
songs.head()

This dataframe looks like (130663, 17) and has features of: ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'popularity']


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0


In [18]:
#There are no nas
songs.isna().sum()
songs.columns

Index(['artist_name', 'track_id', 'track_name', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence',
       'popularity'],
      dtype='object')

In [19]:
df = songs.copy()
df = df.drop(columns=['artist_name', 'track_id', 'track_name'])
df.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'popularity'],
      dtype='object')

In [20]:
steps = [('scaler', StandardScaler()), ('knn', NearestNeighbors(n_neighbors=3, algorithm='auto'))]
pipeline = Pipeline(steps)
trans_df = pipeline.fit(df)

In [69]:
test1 = df.sample(1)
print(test1)
test1 = trans_df[0].transform(test1)
rec_songs = trans_df[1].kneighbors(test1)
rec_songs[1][0]

       acousticness  danceability  duration_ms  energy  instrumentalness  key  \
22659        0.0544         0.777       323810   0.494               0.0   10   

       liveness  loudness  mode  speechiness    tempo  time_signature  \
22659     0.114     -9.77     0       0.0984  125.873               4   

       valence  popularity  
22659     0.35          24  


array([22659, 85426, 43695])

In [68]:
print(songs.iloc[101179])
print(songs.iloc[99694])







artist_name                     Gagan Sran
track_id            5r3hhnAhMgovYagXudUp39
track_name                   Down to Earth
acousticness                         0.185
danceability                         0.693
duration_ms                         215690
energy                               0.888
instrumentalness                         0
key                                      2
liveness                            0.0845
loudness                            -2.128
mode                                     1
speechiness                          0.205
tempo                              173.865
time_signature                           4
valence                              0.716
popularity                               6
Name: 101179, dtype: object
artist_name                 Ajayvir Chhina
track_id            0hxZBIASyEFOiMroOMxJ4b
track_name                English Vinglish
acousticness                         0.232
danceability                         0.684
duration_ms               